In [7]:
# !pip install "vllm[serve]" --quiet
# !pip install pyngrok --quiet
# !pip install  pyyaml --quiet

# @title 1. Install dependencies
!pip install -q \
    vllm \
    pyyaml \
    pyngrok --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.4/326.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5

In [ ]:
TOKEN_HF=""
# Login using Hugging Face token
from huggingface_hub import login

# ⚠️ Replace this with your actual token or use input() to avoid saving it in code
# hf_token = input("🔐 Enter your Hugging Face token: ")
login(TOKEN_HF)


In [9]:
# @title 2. Write `config.yaml` for Mistral AWQ
import yaml

config = {
    "host": "0.0.0.0",
    "port": 8000,
    "uvicorn-log-level": "info",
    # Model & quantization
    "model": "TheBloke/Mistral-7B-Instruct-v0.1-AWQ",
    "quantization": "awq",
    "dtype": "float16",               # T4 supports float16
    "max-model-len": 2048,
    "gpu-memory-utilization": 0.8,
    "tensor-parallel-size": 1
}

with open("config.yaml", "w") as f:
    yaml.dump(config, f, sort_keys=False)

print("✅ config.yaml:")
!cat config.yaml


✅ config.yaml:
host: 0.0.0.0
port: 8000
uvicorn-log-level: info
model: TheBloke/Mistral-7B-Instruct-v0.1-AWQ
quantization: awq
dtype: float16
max-model-len: 2048
gpu-memory-utilization: 0.8
tensor-parallel-size: 1


In [10]:
#@title 3. (Optional) Open an ngrok tunnel so your local PC can hit Colab
from pyngrok import ngrok
ngrok.set_auth_token("2wg0E7kWJrEWTfCtgn4larEEMTo_4cqJUrxwYZBJSQkYN2f7V")  # if you have one
public_url = ngrok.connect(8000, "http")
NGROK_URL=public_url.public_url

print("🔗 Public URL →", public_url)

🔗 Public URL → NgrokTunnel: "https://96c0-34-141-169-157.ngrok-free.app" -> "http://localhost:8000"


In [11]:
public_url.public_url

'https://96c0-34-141-169-157.ngrok-free.app'

In [12]:

# @title 4. Launch vLLM in the background
!pkill -f "vllm serve" 2>/dev/null || true
!nohup vllm serve --config config.yaml &> mistral_awq_server.log & echo "✅ vLLM (Mistral AWQ) starting… logs → mistral_awq_server.log"

^C
✅ vLLM (Mistral AWQ) starting… logs → mistral_awq_server.log


In [16]:
import requests, json

VLLM_URL = f"{NGROK_URL}/v1/completions"

german_chunk="""
das ist ein Thema das an keinem von uns
Vorbeigehen sollte ai agents warum sind
die relevant wenn wir an die
ki-anwendung denken mit dem wir heute
arbeiten wie unterscheidet sich ein ai
Agent von dem was ich in jgpt mache oder
Copilot und was hat das mit meiner
Arbeit zu
[Musik]
"""

prompt = "Translate and segment this German transcript chunk into full sentences (ignore [Musik]):\n\n" + german_chunk

payload = {
    "model": "TheBloke/Mistral-7B-Instruct-v0.1-AWQ",
    "prompt": prompt,
    "max_tokens": 1024,
    "temperature": 0.7,
    "top_p": 0.9,
    "stream": False
}

resp = requests.post(VLLM_URL, json=payload)
resp.raise_for_status()
print(resp.json()["choices"][0]["text"])




"This is a topic that none of us should pass by. AI agents, why are they relevant when we think of the Ki-application with which we work today? How does an AI agent differ from what I do in JGPT or Copilot, and what does that have to do with my work?"


In [15]:
11

11